In [6]:
import pyomo.environ as pyo

## Enter your data here

In [120]:
# parameters
csc = 0.6
dmg_add = 0.5 + 3.154 + 0.24 + 0.495 + 0.24 + 0.784 + 0.396
dmg_bleed = 0.313
dmg_bers_p = 1.484
dmg_vuln_p = 0.396
dmg_csd_i = 0.175+0.175+0.35
paingorger_roll = 2

# passive skill ranks for Heavy Handed (hh), Cut to the Bone (cttb)
ranks_hh = 3
ranks_cttb = 3

## Model starts here

In [121]:
model = pyo.ConcreteModel()

In [122]:
# variable declaration, vd=vulnerable damage, bd=berserking damage, csd = critical strike damage
model.x_vd = pyo.Var(domain=pyo.NonNegativeReals)
model.x_csd = pyo.Var(domain=pyo.NonNegativeReals)
model.x_bd = pyo.Var(domain=pyo.NonNegativeReals)

In [123]:
# combine parameters with variables so the final damage formula is less verbose
pg = 1 + paingorger_roll
hh = 1 + 0.05*ranks_hh
cttb = 1 + 0.05*ranks_cttb

vd = dmg_vuln_p + model.x_vd
csd = dmg_csd_i + model.x_csd
bd = dmg_bers_p + model.x_bd

In [124]:
# subexpressions for the objective function

# subexpression for the hit damage part of bash, assuming Adaptability and Moonrise(1.8) do not interact with Berserk Ripping
# assuming vulnerable damage works with Paingorger, but crit does not
# 1.2 vuln multiplier, 1.15 Two-Handed Mace Expertise
bash_hit = 1.8 * 1.2 * (1+bd*0.1) * ((1-csc+pg)*(1+dmg_add+vd+bd) + csc*1.5*hh*1.15*(1+dmg_add+vd+csd+bd))

# subexpression for the bleed caused by Berserk Ripping
# multipliers that apply to both normal bleeds and gushing wounds bleeds already included in the base
br_base = (1+pg) * 0.6 * 1.2 * (1+vd*0.15) * (1+bd*0.1) * (1+dmg_add+dmg_bleed+vd+bd) * cttb
bash_bleed = (1-csc)*br_base + csc*br_base*((1+csd)*(1+0.5*1.4)*hh*1.15)

In [125]:
# preliminary stat budget of 500+500+150
stat_budget = 11.5

In [126]:
# objective function maximizes the total multiplier (excluding global multipliers that apply to both hit and bleed independent of variables)
model.obj = pyo.Objective(expr = bash_hit+bash_bleed, sense=pyo.maximize)

# constraints for the stat budget
model.statbudget = pyo.Constraint(expr = model.x_vd+model.x_csd+model.x_bd <= stat_budget)
# artificially limit berserking damage to 300
model.bd = pyo.Constraint(expr = bd <= 3)

In [127]:
model.pprint()

3 Var Declarations
    x_bd : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    x_csd : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    x_vd : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 0.4*(2.88*(1 + (0.396 + x_vd)*0.15)*(1 + (1.484 + x_bd)*0.1)*(0.396 + x_vd + 7.122 + 1.484 + x_bd)*1.15) + 0.6*(2.88*(1 + (0.396 + x_vd)*0.15)*(1 + (1.484 + x_bd)*0.1)*(0.396 + x_vd + 7.122 + 1.484 + x_bd)*1.15)*((0.7 + x_csd + 1)*1.7*1.15*1.15) + 2.16*(1 + (1.484 + x_bd)*0.1)*(3.4*(0.396 + x_vd + 6.809 + 1.484 + x_bd) + 1.19024999999999

In [139]:
opt = pyo.SolverFactory('couenne', executable = 'C:/Users/Daniel/idaes_opti_venv/solvers/couenne.exe')
results = opt.solve(model, tee=True)

Couenne 0.5.8 -- an Open-Source solver for Mixed Integer Nonlinear Optimization
Mailing list: couenne@list.coin-or.org
Instructions: http://www.coin-or.org/Couenne
couenne: 
ANALYSIS TEST: Couenne: new cutoff value -1.3561195767e+03 (0.01 seconds)
NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I             1         OPT -1356.1196        6 0.001
Couenne: new cutoff value -1.3561195942e+03 (0.011 seconds)
Loaded instance "C:\Users\Daniel\AppData\Local\Temp\tmp97v4tftu.pyomo.nl"
Constraints:            2
Variables:              3 (0 integer)
Auxiliaries:           10 (0 integer)

Coin0506I Presolve 19 (-4) rows, 9 (-4) columns and 59 (-9) elements
Clp0006I 0  Obj -4447.0338 Primal inf 1094.7245 (6)
Clp0006I 14  Obj -2151.9056
Clp0000I Optimal - objective value -2151.9056
Clp0032I Optimal objective -2151.905553 - 14 iterations time 0.002, Presolve 0.00
Clp0000I Optimal - objective value -2151.9056
Cbc0012I Integer solution of -1

In [149]:
if (results.solver.status == pyo.SolverStatus.ok) and (results.solver.termination_condition == pyo.TerminationCondition.optimal):
    print('Found optimal solution, with the following variable values:')
else:
    print('No optimal solution found, but these are the best variable assignments:')

print('Total Vulnerable Damage:', model.x_vd.value+dmg_vuln_p)
print('Total (raw) Critical Strike Damage:', model.x_csd.value+dmg_csd_i)
print('Total Berserking Damage:', model.x_bd.value+dmg_bers_p)

print('Ratio of CSD to VD:', (model.x_vd.value+dmg_vuln_p)/(model.x_csd.value+dmg_csd_i))

Found optimal solution, with the following variable values:
Total Vulnerable Damage: 5.780965609422134
Total (raw) Critical Strike Damage: 5.299034390577865
Total Berserking Damage: 2.9999999999999996
Ratio of CSD to VD: 1.0909469883232281


In [130]:
print(model.x_csd.value)

4.599208121466778


In [131]:
print(model.x_vd.value)

5.384791978382899


In [132]:
print(model.x_bd.value)

1.5159999999999996


In [133]:
print(pyo.value(model.obj))

1356.1195942269321
